## RLHF（Reinforcement Learning from Human Feedback）

- **目标**：让模型更符合人类意图、更安全、更有用
- **核心思想**：
  - 用监督微调（SFT）教会模型基本的指令跟随
  - 用偏好数据训练奖励模型（RM），学会打分“更好/更差”的回答
  - 用强化学习（PPO）在奖励信号下优化策略，权衡质量、稳定性与多样性
- **关键组件**：指令数据、偏好数据（A/B 对比）、奖励模型、强化学习算法、KL 约束/参考策略
- **典型产物**：
  - SFT 模型（会做事）
  - RM 奖励模型（会打分）
  - PPO 后的对齐模型（做得更好）
  - DPO（取代 RM+PPO 的直接偏好优化）

### RLHF 的三阶段流程（工程化视角）

| 阶段 | 名称 | 作用 | 技术 |
|---|---|---|---|
| 1️⃣ | SFT（监督微调） | 教模型执行指令 | CrossEntropyLoss |
| 2️⃣ | 奖励模型（RM）训练 | 学会“什么样的回答更好” | Pairwise ranking (A > B) |
| 3️⃣ | PPO 强化优化 | 用奖励信号优化生成策略 | PPO 算法（Policy Gradient） |



### 3️⃣ PPO 强化优化
- **输入**：SFT 模型作为初始策略 \(\pi_\theta\)，奖励模型 r 作为奖励信号
- **目标**：在 KL 约束下最大化期望奖励，提升对齐度与有用性
- **训练**：PPO（剪切策略梯度），引入 KL 惩罚以保持与参考策略接近
- **输出**：PPO 后的对齐模型（更符合人类偏好）
- **实践要点**：高质量偏好数据与稳定的 KL 控制是成功关键；监控长度偏置、模式坍缩与过拟合。

### DPO（Direct Preference Optimization）
- **定位**：作为第 3 阶段（PPO）的常见替代方案，用偏好对直接优化策略。
- **核心**：基于 \((x, y_{pos}, y_{neg})\) 提高 \(y_{pos}\) 概率、降低 \(y_{neg}\)，并以参考策略 \(\pi_{ref}\) 的对数概率差作隐式 KL 约束。
- **直观目标**：最小化 \(-\log \sigma\big(\beta[(\log \pi_\theta(y_{pos}|x) - \log \pi_\theta(y_{neg}|x)) - (\log \pi_{ref}(y_{pos}|x) - \log \pi_{ref}(y_{neg}|x))]\big)\)
- **优点**：流程简单、无奖励模型与 RL 回路、稳定易复现、吞吐高。
- **局限**：依赖高质量偏好数据；极端分布迁移下可控性较弱。

### 实验设置：模型与数据集选择
- **模型**：Qwen2.5-1.5B-Instruct（中文指令能力强，小参数、易于 LoRA/QLoRA）
- **SFT 数据**：BelleGroup/train_0.5M_CN（中文指令-回答对，体量适中，可采样）
- **偏好数据（用于 DPO/RM）**：argilla/ultrafeedback-binarized-preferences（成对偏好，易直接用于 DPO）

## 环境安装

In [ ]:
# 安装需要的库
%env TOKENIZERS_PARALLELISM=false
%pip install -q torch torchvision torchaudio
%pip install -q "transformers>=4.44.0" "datasets>=2.18.0" "accelerate>=0.33.0" \
                "peft>=0.12.0" "trl>=0.9.6" "sentencepiece>=0.1.99" "safetensors>=0.4.5" \
                "huggingface_hub>=0.24.0" "modelscope>=1.14.0" "protobuf>=4.25.0" \
                "numpy>=1.24.0" "scipy>=1.10.0" "tiktoken>=0.7.0"

# 仅在 CUDA 可用时安装 bitsandbytes（可选）
import torch
if torch.cuda.is_available():
    %pip install -q "bitsandbytes>=0.43.0"

# 打印关键版本，便于排查
import importlib.metadata as im
v = lambda n: (im.version(n) if n in {d.metadata['Name'] for d in im.distributions()} else 'N/A')
print("[Versions]",
      "torch=", v("torch"),
      "transformers=", v("transformers"),
      "datasets=", v("datasets"),
      "accelerate=", v("accelerate"),
      "peft=", v("peft"),
      "trl=", v("trl"),
      "modelscope=", v("modelscope"),
      "sentencepiece=", v("sentencepiece"),
      "bitsandbytes=", v("bitsandbytes"))


## 模型下载

In [ ]:
# 下载Qwen/Qwen2.5-1.5B-Instruct
import torch
from modelscope.hub.snapshot_download import snapshot_download
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "Qwen/Qwen2.5-1.5B-Instruct"  # ModelScope 上的模型标识（公共可直接下载）

# 选择设备
device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")

# 通过 ModelScope 下载到本地缓存，然后用 Transformers 从本地目录加载
model_dir = snapshot_download(model_id)

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_dir, trust_remote_code=True)
model = model.to(device)

print(f"[Device] {device}")

# 简单自检
txt = "你好，简要介绍一下你自己。"
inputs = tokenizer(txt, return_tensors="pt").to(device)
with torch.inference_mode():
    out = model.generate(**inputs, max_new_tokens=64, do_sample=False)
print(tokenizer.decode(out[0], skip_special_tokens=True))

## 数据集下载

In [2]:
# 下载sft数据集和偏好数据集
from modelscope.hub.snapshot_download import snapshot_download

# 指定两个数据集名称（可按需修改）
sft_id = "AI-ModelScope/train_0.5M_CN"  # 中文数据集
pref_id_primary = "HuggingFaceH4/ultrafeedback_binarized" # 多语，英文为主，可筛出中文子集
pref_id_fallback = "AI-ModelScope/ultrafeedback-binarized-preferences-cleaned-kto" # UltraFeedback 兼容格式，亦可筛中文
 
# 仅使用 ModelScope 下载到本地缓存（不做回退）
sft_dir = snapshot_download(sft_id, repo_type="dataset")
pref_dir = snapshot_download(pref_id_primary, repo_type="dataset")
pref_dir_fallback = snapshot_download(pref_id_fallback, repo_type="dataset")

2025-11-01 23:09:18,235 - modelscope - INFO - Fetching dataset repo file list...


2025-11-01 23:09:19,679 - modelscope - INFO - Fetching dataset repo file list...


2025-11-01 23:09:21,564 - modelscope - INFO - Fetching dataset repo file list...


In [13]:
# 加载预览数据
from datasets import load_dataset
import pandas as pd

pd.set_option("display.max_colwidth", None)

# 将嵌套结构规范为可读文本
def normalize_text(x):
    if x is None:
        return None
    if isinstance(x, str):
        return x
    if isinstance(x, dict):
        return x.get("content") or x.get("text") or x.get("value") or str(x)
    if isinstance(x, (list, tuple)):
        parts = []
        for it in x:
            if isinstance(it, str):
                parts.append(it)
            elif isinstance(it, dict):
                role = it.get("role")
                content = it.get("content") or it.get("text") or it.get("value")
                if content is not None:
                    parts.append((f"{role}: " if role else "") + str(content))
            else:
                parts.append(str(it))
        return "\n---\n".join(parts)
    return str(x)

# SFT：从本地目录加载并展示前 2 条（表格化，完整文本）
sft_preview = load_dataset(sft_dir, split="train[:2]")
sft_rows = []
for i in range(len(sft_preview)):
    ex = sft_preview[i]
    row = {
        "instruction": normalize_text(ex.get("instruction")),
        "input": normalize_text(ex.get("input")),
        "output": normalize_text(ex.get("output")),
        "instr_len": len(str(ex.get("instruction", ""))),
        "input_len": len(str(ex.get("input", ""))),
        "output_len": len(str(ex.get("output", ""))),
    }
    sft_rows.append(row)
print(f"[SFT] 预览 {len(sft_rows)} 条 / split=train[:2]")
display(pd.DataFrame(sft_rows))

# 偏好：分别加载主源和备用数据集并展示

# 辅助函数：加载偏好数据
def load_pref_data(ds_dir, source_name):
    # 尝试多种加载方式
    for split_name in ["train_prefs[:2]", "train[:2]"]:
        try:
            return load_dataset(ds_dir, split=split_name), f"{source_name} ({split_name.split('[')[0]})"
        except Exception:
            continue
    # 如果都失败，尝试直接从 data 目录加载（支持 Parquet 和 JSONL）
    try:
        import os
        import json
        import glob
        data_dir = os.path.join(ds_dir, "data")
        if os.path.exists(data_dir):
            # 优先查找 Parquet 文件
            parquet_files = glob.glob(os.path.join(data_dir, "*.parquet"))
            if parquet_files:
                from datasets import Dataset
                # 读取第一个 parquet 文件
                dataset = Dataset.from_parquet(parquet_files[0])
                # 只取前2行
                dataset = dataset.select(range(min(2, len(dataset))))
                return dataset, f"{source_name} (from Parquet)"
            # 查找 JSONL 文件
            jsonl_files = glob.glob(os.path.join(data_dir, "*.jsonl"))
            if jsonl_files:
                # 读取前2行
                data = []
                with open(jsonl_files[0], 'r', encoding='utf-8') as f:
                    for i, line in enumerate(f):
                        if i >= 2:
                            break
                        try:
                            data.append(json.loads(line))
                        except:
                            pass
                if data:
                    from datasets import Dataset
                    dataset = Dataset.from_list(data)
                    return dataset, f"{source_name} (from JSONL)"
    except Exception as e:
        print(f"[Error] 加载失败: {e}")
        import os
        if os.path.exists(ds_dir):
            print(f"[Debug] 目录存在，文件列表: {os.listdir(ds_dir)[:10]}")
    return None, None

# 辅助函数：格式化偏好数据
def format_pref_data(pref_preview, source_name):
    if pref_preview is None:
        print(f"[Preference {source_name}] 无法加载")
        return
    pref_rows = []
    for i in range(len(pref_preview)):
        ex = pref_preview[i]
        prompt = ex.get("prompt") or ex.get("question") or ex.get("instruction") or ex.get("input")
        y_pos = ex.get("chosen") or ex.get("better_response") or ex.get("pos") or ex.get("preferred") or ex.get("y_pos")
        y_neg = ex.get("rejected") or ex.get("worse_response") or ex.get("neg") or ex.get("other") or ex.get("y_neg")
        
        # 处理特殊格式：如果有 completion + annotations，可能需要提取
        # 这里先简单显示，后续训练时再做转换
        prompt = normalize_text(prompt)
        y_pos = normalize_text(y_pos)
        y_neg = normalize_text(y_neg)
        
        row = {
            "prompt": prompt,
            "y_pos": y_pos,
            "y_neg": y_neg,
            "prompt_len": len(str(prompt or "")),
            "y_pos_len": len(str(y_pos or "")),
            "y_neg_len": len(str(y_neg or "")),
        }
        pref_rows.append(row)
    print(f"[Preference {source_name}] 预览 {len(pref_rows)} 条")
    display(pd.DataFrame(pref_rows))

# 加载主源
pref_preview_primary, pref_source_primary = load_pref_data(pref_dir, "Primary")
format_pref_data(pref_preview_primary, "Primary")

# 加载备用
pref_preview_fallback, pref_source_fallback = load_pref_data(pref_dir_fallback, "Fallback")
format_pref_data(pref_preview_fallback, "Fallback")




[SFT] 预览 2 条 / split=train[:2]


,instruction,input,output,instr_len,input_len,output_len
0,给定一个英文句子，翻译成中文。\nI love to learn new things every day.\n,,我每天喜欢学习新事物。,54,0,11
1,给定一个文字输入，将其中的所有数字加1。\n“明天的会议在9点开始，记得准时到达。”\n,,“明天的会议在10点开始，记得准时到达。”,42,0,21


[Preference Primary] 预览 2 条


,prompt,y_pos,y_neg,prompt_len,y_pos_len,y_neg_len
0,how can i develop a habit of drawing daily,"user: how can i develop a habit of drawing daily\n---\nassistant: Developing a daily habit of drawing can be challenging but with consistent practice and a few tips, it can become an enjoyable and rewarding part of your daily routine. Here are some strategies to help you develop the habit of drawing daily:\n\n1. Set a specific time: Allocate a specific time of the day to draw. It could be in the morning, afternoon, or evening. Make drawing a part of your daily routine.\n2. Set a specific duration: Determine the amount of time you want to spend on drawing each day. It can be as little as 10 minutes or as long as an hour. Be consistent with the duration to help build the habit.\n3. Start small and simple: Don't try to create a masterpiece every day, start with simple and easy-to-do sketches. Focus on improving your skills gradually.\n4. Use a variety of tools and mediums: Experiment with different tools like pencils, pens, markers, and different mediums like paper, canvas, or digital apps to keep your drawing practice interesting and engaging.\n5. Take breaks and rest: Taking breaks and resting after some time of drawing can help you avoid burnout and stay motivated.\n6. Challenge yourself: Set challenges like drawing objects from memory or a specific subject to improve your skills and keep your drawing practice interesting.\n7. Track your progress: Keep a record of your daily drawing practice and track your progress. This can be a source of motivation and help you see how far you've come.\n\nRemember, developing a habit takes time and patience. Stay consistent with your drawing practice, be flexible and open to trying new things, and with time, you'll develop a habit of daily drawing that brings you joy and satisfaction.","user: how can i develop a habit of drawing daily\n---\nassistant: As an AI language model, I cannot personally develop habits for you. But, here are some tips for developing a habit of drawing daily:\n\n1. Start small: Start with simple drawings or doodles and gradually increase the complexity of your drawings.\n\n2. Set a schedule: Set a specific time every day to draw, and try to stick to it as much as possible.\n\n3. Make it fun: Don't pressure yourself to create masterpieces every time you draw. Make it a relaxing and enjoyable experience.\n\n4. Use resources: There are many drawing tutorials available online. Use resources like YouTube or online drawing courses to help you improve your skills.\n\n5. Surround yourself with inspiration: Expose yourself to a variety of art forms, such as paintings, illustrations, and photographs, to inspire and motivate you.\n\nRemember, everyone has their own creative style and pace. Just keep practicing and enjoying the process of drawing.",42,1737,977
1,how can I transform the getPosition method of antv/g's group in zrender?,"user: how can I transform the getPosition method of antv/g's group in zrender?\n---\nassistant: It is not recommended to modify built-in methods as it can lead to unexpected results and potential bugs. You may consider developing a new method or exploring other methods to achieve your desired outcome. Alternatively, you can search for other libraries or modules that offer similar functionalities or reach out to the library's support team for assistance.","user: how can I transform the getPosition method of antv/g's group in zrender?\n---\nassistant: Thank you for reaching out for assistance! I'm here to help you with your question. However, I must point out that the question itself may not be meaningful.\n\nThe `getPosition` method is a part of the AntV/G library, which is a gradient boosting framework. It is not clear what you mean by ""transforming"" this method, as it is not a functional programming concept. Additionally, the concept of ""zrender"" is not related to AntV/G.\n\nCould you please provide more context or clarify your question? I'd be happ

[Preference Fallback] 预览 2 条


,prompt,y_pos,y_neg,prompt_len,y_pos_len,y_neg_len
0,Question: Combine facts and answer this: For what team does the 17th pick of the NHL draft that included Jack Campbell and Brock Nelson play professionally?\nAnswer:,None,None,164,0,0
1,Question: Combine facts and answer this: For what team does the 17th pick of the NHL draft that included Jack Campbell and Brock Nelson play professionally?\nAnswer:,None,None,164,0,0


## 1️⃣ SFT（监督微调）
- **输入**：指令-回答对（高质量、人类书写/筛选）
- **目标**：让模型基本学会“按指令作答”
- **训练**：最小化交叉熵损失（参考常用指令数据集）
- **输出**：SFT 模型（作为后续 RM/PPO 的参考策略）

In [ ]:
# 加载sft数据集 dataset
from datasets import Dataset
import os
import json

sft_path = os.path.expanduser(sft_dir)

# 检查目录内容
print("[文件列表]", os.listdir(sft_path))

# 尝试找到 JSON 或 JSONL 文件
json_files = [f for f in os.listdir(sft_path) if f.endswith(".json") or f.endswith(".jsonl")]
if not json_files:
    raise RuntimeError(f"在 {sft_path} 未找到 JSON/JSONL 文件，请手动查看文件结构。")

json_path = os.path.join(sft_path, json_files[0])
print(f"[加载文件] {json_path}")

# 加载 JSON 数据
data = []
with open(json_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except:
            pass

# 转换成 HuggingFace Dataset
sft_ds = Dataset.from_list(data)
print(f"[SFT] 成功加载 {len(sft_ds)} 条样本。")
print("[示例样本]", sft_ds[0])


In [ ]:
# 数据集切分 按比例划分，比如 98% 训练、2% 验证
sft_split = sft_ds.train_test_split(test_size=0.02, seed=42)

# 重构数据格式：构造 prompt-completion 结构（符合 TRL 0.24 标准）
def _to_sft(example):
    instr = example.get("instruction", "")
    inp = example.get("input", "")
    output = example.get("output", None)
    # 使用空格结尾，避免分词边界问题
    prompt = (instr + ("\n" + inp if inp else "")).strip() + " "
    return {"prompt": prompt, "completion": output}

# 切分并格式化
train_ds = sft_split["train"].map(_to_sft, remove_columns=sft_split["train"].column_names)
eval_ds = sft_split["test"].map(_to_sft, remove_columns=sft_split["test"].column_names)

print("训练集样本数:", len(train_ds))
print("验证集样本数:", len(eval_ds))
print("[示例]", train_ds[0])


In [ ]:
# 配置用于 QLoRA 训练下的模型与 tokenizer 初始化

base_model_or_dir = model_dir  # 复用 ModelScope 下载的模型目录
output_dir = "outputs/sft_qlora"
max_seq_length = 2048

use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

# bitsandbytes QLoRA 配置
quantization_config = None
if use_cuda:
    try:
        from transformers import BitsAndBytesConfig
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
        print("[QLoRA] 使用 bitsandbytes 4-bit 量化加载模型")
    except Exception as e:
        print(f"[Warn] 未启用4bit量化：{e}")
else:
    print("[Info] 当前为非 CUDA 环境，使用常规精度加载")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_or_dir, use_fast=True, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


In [ ]:
# 定义格式化函数：构造训练文本模板
# 注意：TRL 0.24.0 的 formatting_func 接收单个 dict，返回单个字符串
def formatting_func(example):
    """将 prompt-response 对转换为模型输入文本（单条处理）"""
    prompt = str(example["prompt"]).strip()
    resp = str(example["response"]).strip()
    # 拼接格式：符合 Qwen 的模板
    text = f"<|user|>\n{prompt}\n<|assistant|>\n{resp}{tokenizer.eos_token}"
    return text

print("[格式化函数已定义]")
print("[示例]", formatting_func({"prompt": "你好", "response": "你好！有什么可以帮助您的吗？"}))


In [ ]:
# LoRA 配置与训练参数
from peft import LoraConfig
from transformers import TrainingArguments

# LoRA 配置：低秩适配器参数
lora_config = LoraConfig(
    r=16,                    # 秩（rank），控制适配器矩阵的维度，越大容量越大但参数量更多（常用 8/16/32）
    lora_alpha=32,          # 缩放系数，与 r 成比例使用（通常 alpha=2*r）
    lora_dropout=0.05,      # 适配器层 dropout（防止过拟合，常用 0.05-0.1）
    bias="none",            # 是否训练 bias（"none"/"all"/"lora_only"）
    task_type="CAUSAL_LM",   # 任务类型：因果语言模型（生成任务）
)

# 训练参数：控制训练流程与优化
args = TrainingArguments(
    # 输出与保存
    output_dir=output_dir,              # 模型/日志输出目录
    save_steps=200,                     # 每 N 步保存一次 checkpoint
    save_total_limit=2,                 # 最多保留 N 个 checkpoint（避免占磁盘）
    save_safetensors=True,              # 使用 safetensors 格式保存（更快更安全）
    
    # 批大小与梯度
    per_device_train_batch_size=1,      # 每设备训练 batch 大小（显存受限时先用 1）
    per_device_eval_batch_size=1,       # 每设备验证 batch 大小
    gradient_accumulation_steps=8,      # 梯度累积步数（等效 batch = 1 * 8 = 8）
    
    # 训练轮次与学习率
    num_train_epochs=1,                 # 训练轮次数
    learning_rate=2e-4,                 # 初始学习率（LoRA 常用 1e-4 到 5e-4）
    lr_scheduler_type="cosine",         # 学习率调度器（余弦退火）
    warmup_ratio=0.03,                  # warmup 比例（前 3% 步数线性增长 LR）
    
    # 评估与日志
    eval_strategy="steps",              # 评估策略（"steps"/"epoch"/"no"）
    eval_steps=100,                     # 每 N 步评估一次
    logging_steps=10,                   # 每 N 步打印一次日志
    report_to=["none"],                 # 不向外部上报（可改为 ["wandb"] 等）
    
    # 模型检查点
    load_best_model_at_end=True,        # 训练结束加载最佳模型
    metric_for_best_model="eval_loss",  # 最佳模型指标
    greater_is_better=False,            # 该指标越小越好
    
    # 性能优化
    bf16=use_cuda,                      # CUDA 上用 bfloat16（算力与稳定性平衡）
    fp16=False,                         # 禁用 FP16（避免冲突）
    gradient_checkpointing=True,        # 梯度检查点（牺牲时间换显存）
)

print("训练参数配置完成")

In [9]:
# 训练器的构造
from trl import SFTTrainer

# 加载预训练模型（QLoRA 模式下自动应用量化）
model = AutoModelForCausalLM.from_pretrained(
    base_model_or_dir,
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map="auto" if use_cuda else None,
)

# 构造 SFT 训练器
trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    peft_config=lora_config,
    args=args,
    # 不使用 formatting_func，直接用 prompt-completion 结构
)

In [ ]:
# 启动训练
trainer.train()

# === 保存适配器（LoRA 权重）===
adapter_dir = os.path.join(output_dir, "adapter")
trainer.model.save_pretrained(adapter_dir)
print(f"[Done] SFT + QLoRA 训练完成，LoRA 权重已保存至: {adapter_dir}")


## 2️⃣ 奖励模型（RM）训练
- **输入**：同一指令下成对回答（A、B），以及偏好标签（A > B）
- **目标**：学习“偏好评分函数” r(x, y)
- **训练**：Pairwise ranking（如 Bradley–Terry/Logistic loss）
- **输出**：能对任意回答打分的奖励模型

In [ ]:
## RM 训练
# 加载偏好数据集
from datasets import load_dataset
import pandas as pd

# 指定数据集名称
